## b) Dos funciones auxiliares

- b.1) Una primera función convertirá un número romano en un entero.

- b.2) Una segunda función que nos diga qué líneas hacen referencia a categorías agrupadas de causas de muerte.


### b.1) Conversor de dígitos romanos a enteros árabes.

Definimos nuestra función:

In [12]:
def romano_a_entero(num):
    
    """
    Función que transforma string interpretados como números romanos a números enteros desde el 1 hasta el 1000.
        
    Parameters
    ----------
    num : string
        String que representa un número romano.
    
    Returns
    -------
    valor: int
        Equivalente entero al número romano.
    string
        Si el número introducido no es un numeral romano nos lo indicará.
    
    
    Example
    -------
    >>> romano_a_entero("IV")
    4
    >>> romano_a_entero("CXXIX")
    129
    """

    # Definimos un diccionario con los dígitos romanos. Observar que como tienen una lógica aditiva y sustractiva, 
    # es más útil definir los que provienen de lógica sustractiva por separado en el rango 1(I)-1000(M)
    
    romanos = {'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900, 'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000}
    
    # Inicializamos algunas variables:
    
    i = 0
    valor = 0
    
    # Lanzamos un bucle de lectura, obsérvese el proceso, lee bloques de 2 en dos y determina si ese bloque está en 
    # nuestro diccionario de dígitos, en caso de que lo esté, lo interpreta con el valor indicado por el diccionario
    # y lo añade al valor acumulado previo y continúa con la lectura del string en el valor posterior a ese dígito.
    # Si no lo está, coge únicamente el primero de ellos, lo interpreta, lo añade y continúa desde el siguiente a ese.
    
    # En caso de que un valor de entrada no sea un dígito romano, nos devuelve un mensaje que nos lo indica.
    
    while i < len(num):
        if num[i:i+2] in romanos:
            valor = valor + romanos[num[i:i+2]]
            i = i+2
            
        else:
            if num[i] in romanos:
                valor = valor + romanos[num[i]]
                i = i+1
            else:
                return("El valor a interpretar no es un número romano")
            
    return(valor)

Podemos probar nuestra función con alguna colección de valores:

In [20]:
print([(r, romano_a_entero(r)) \
      for r in ["I", "IV", "XIV", "XXXIX", "VL", "LXIV", "MCDXCII", "XXG"]])

[('I', 1), ('IV', 4), ('XIV', 14), ('XXXIX', 39), ('VL', 55), ('LXIV', 64), ('MCDXCII', 1492), ('XXG', 'El valor a interpretar no es romano')]


### b.2) Detector de categorías de muertes agrupadas.

Las condiciones que debe cumplir nuestra función son las siguientes:

Estas líneas empiezan por tres  dígitos, seguidos por un guion y otros tres dígitos:

        001-008 I.Enfermedades infecciosas y parasitarias Hombres De 65 a 69 años 2001 305

También hay algunas líneas que totalizan **todas** las causas:

        001-102 I-XXII.Todas las causas Mujeres De 10 a 14 años 1983 308

Nos interesan las primeras, pero no las segundas.

La función debe detectar filas cuyo valor en la columna "Causa de muerte" cumpla las siguiente condiciones:

        - Empieza por tres dígitos, seguidos por un guion y otros tres dígitos.
        - No totaliza **todas** las causas de fallecimiento.
        
Algunas de las categorías de muertes figuran de la siguiente manera:

        082  XVI.Afecciones originadas en el periodo perinatal
        
Estas categorías no cumplen las condiciones previas y, por lo tanto, no serán incluidas. En apartados posteriores del proyecto se nos pide incluirlas para la elaboración de cierto diccionario por lo que se utilizará una modificación de esta misma función de comprobación. 

Utilizaremos funciones regulares del paquete "re" para hacer estas comprobaciones.

Vamos allá:

In [17]:
import re

def es_grupo_y_no_total(causa):
    
    """
    Función que comprueba si un determinado string cumple las siguientes condiciones:
    
       - Empieza por tres dígitos, seguidos por un guion y otros tres dígitos.
       - No totaliza **todas** las causas de fallecimiento.
    
    Que quedan sintetizadas en las varaibles "patron" y "patron2" definidas durante la función.
        
    Parameters
    ----------
    causa : string
        String con la causa de muerte, a analizar para ver si cumple las condiciones.
    
    Returns
    -------
    Un valor boolean indicando si cumple las condiciones (True) o no (False).
    
    
    Example
    -------
    >>> es_grupo_y_no_total("001-008 I.Enfermedades infecciosas y parasitarias")
    True
    >>> es_grupo_y_no_total("001-102 I-XXII.Todas las causas Mujeres De 10 a 14 años 1983 308")
    False
    >>> es_grupo_y_no_total("XV.Embarazo")
    False
    """
    
    # Definimos nuestras expresiones a reconocer, la primera es una expresión regular que quiere decir:
    # 3 dígitos, seguidos por un guión y otros 3 dígitos.

    patron = "\d{3}-\d{3}"
    patron2 = "Todas las causas"

    # Lanzamos un bucle que busca, en primer lugar, si el string comienza con el patrón: "\d{3}-\d{3}".
    # En caso de que lo cumpla, comprueba si el string contiene el string prohibido: "Todas las causas".
    
    if re.match(patron, causa) != None:
        
        if re.search(patron2, causa) != None:
            return(False)
    
        else:
            return(True)

    else:
        return(False)

Podemos realizar comprobaciones de esta función con diferentes strings que podríamos en contrar en la columna "Causas de muerte" de nuestro DF:

In [18]:
lista_de_causas = [
    "009-041  II.Tumores",
    "009  Tumor maligno del esófago",
    "001-102  I-XXII.Todas las causas",
    "077-080  XIV.Enfermedades del sistema genitourinario",
    "082  XVI.Afecciones originadas en el periodo perinatal",
    "050-052  VI-VIII.Enfermedades del sistema nervioso y de los órganos de los sentidos"
]

for causa in lista_de_causas:
    print(causa.ljust(80), "\t", es_grupo_y_no_total(causa))

009-041  II.Tumores                                                              	 True
009  Tumor maligno del esófago                                                   	 False
001-102  I-XXII.Todas las causas                                                 	 False
077-080  XIV.Enfermedades del sistema genitourinario                             	 True
082  XVI.Afecciones originadas en el periodo perinatal                           	 False
050-052  VI-VIII.Enfermedades del sistema nervioso y de los órganos de los sentidos 	 True


Al poner a trabajar esta función con una lista de strings, como por ejemplo, la columna "Causas de muerte" de nuestro DataFrame con el que estamos trabajando, podremos elaborar una lista de valores booleanos de la misma longitud que nos permita utilizarla como máscara para filtrar y seleccionar las filas que cumplan la condición. Esto se realizará en el siguiente apartado, por lo que considero que no es necesario realizar más ejemplos de prueba de momento.